In [25]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, DataFrames, Plots, Statistics

In [26]:
## Function to Load Data.
function load_data(fname)
    data = DataFrame(CSV.File(fname, header=false)) |> DataFrame;
    return data
end;

In [27]:
hospital_mapping = CSV.read("../data/scaled_up_70/hospital_mapping.csv", DataFrame, pool=true);
distance_cost_vehicle_type_mapping = CSV.read("../data/scaled_up_70/distance_cost_vehicle_type_mapping.csv", DataFrame, pool=true);
fixed_cost_vehicle_type_mapping = CSV.read("../data/scaled_up_70/fixed_cost_vehicle_type_mapping.csv", DataFrame, pool=true);
vehicle_depot_mapping = CSV.read("../data/scaled_up_70/vehicle_depot_mapping.csv", DataFrame, pool=true);
distance_mapping = CSV.read("../data/scaled_up_70/distance_mapping.csv", DataFrame, pool=true);
vehicle_type = CSV.read("../data/scaled_up_70/vehicle_type.csv", DataFrame, pool=true);

In [28]:
vehicle_type

Row,vehicle_num,type
,Int64,Int64
1,1,1
2,2,1
3,3,2
4,4,1
5,5,2
6,6,1
7,7,2
8,8,1
9,9,1


In [29]:
hospital_mapping = hospital_mapping[:,2:end]

Row,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
6,0,1,1,1,1,0,0,1,0,0,0,0,2,1,0,1,3,0,0,0,0,1,1,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,1,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
patient_hospital_mapping = []
max_transfers = maximum(Matrix(hospital_mapping))
for i=1:max_transfers
    transfers = sort(Tuple.(findall(>=(i), Matrix(hospital_mapping))))
    for item in transfers
        push!(patient_hospital_mapping, item)
    end
#    push!(patient_hospital_mapping, sort(Tuple.(findall(>=(i), Matrix(hospital_mapping))))[0])
end

In [31]:
df_patient_hospital_mapping = DataFrame(patient_hospital_mapping)

Row,1,2
,Int64,Int64
1,2,6
2,2,8
3,2,13
4,2,17
5,3,6
6,3,8
7,3,13
8,3,17
9,4,6


In [32]:
# CSV.write("../data/new_formulation/df_patient_hospital_mapping.csv", df_patient_hospital_mapping)

In [33]:
distance_cost_vehicle_type_mapping = distance_cost_vehicle_type_mapping[:,2:end]

Row,cost_per_km
,Float64
1,0.01
2,0.002


In [34]:
fixed_cost_vehicle_type_mapping = fixed_cost_vehicle_type_mapping[:,2:end]

Row,fixed_cost
,Int64
1,100
2,200


In [35]:
vehicle_depot_mapping = vehicle_depot_mapping[:,2:end]

Row,depot_num
,Int64
1,1
2,1
3,1
4,1
5,2
6,1
7,2
8,2
9,1


In [36]:
distance_mapping = distance_mapping[:,3:end]

Row,Depot 1,Depot 2,Patient 1,Patient 2,Patient 3,Patient 4,Patient 5,Patient 6,Patient 7,Patient 8,Patient 9,Patient 10,Patient 11,Patient 12,Patient 13,Patient 14,Patient 15,Patient 16,Patient 17,Patient 18,Patient 19,Patient 20,Patient 21,Patient 22,Patient 23,Patient 24,Patient 25,Patient 26,Patient 27,Patient 28,Patient 29,Patient 30,Patient 31,Patient 32,Patient 33,Patient 34,Patient 35,Patient 36,Patient 37,Patient 38,Patient 39,Patient 40,Patient 41,Patient 42,Patient 43,Patient 44,Patient 45,Patient 46,Patient 47,Patient 48,Patient 49,Patient 50,Patient 51,Patient 52,Patient 53,Patient 54,Patient 55,Patient 56,Patient 57,Patient 58,Patient 59,Patient 60,Patient 61,Patient 62,Patient 63,Patient 64,Patient 65,Patient 66,Patient 67,Patient 68,Patient 69,Patient 70,Dest Patient1,Dest Patient2,Dest Patient3,Dest Patient4,Dest Patient5,Dest Patient6,Dest Patient7,Dest Patient8,Dest Patient9,Dest Patient10,Dest Patient11,Dest Patient12,Dest Patient13,Dest Patient14,Dest Patient15,Dest Patient16,Dest Patient17,Dest Patient18,Dest Patient19,Dest Patient20,Dest Patient21,Dest Patient22,Dest Patient23,Dest Patient24,Dest Patient25,Dest Patient26,Dest Patient27,Dest Patient28,⋯
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,0,12165,8451,8451,8451,8451,7537,7537,7537,7537,8189,8189,9040,9040,8263,8263,8263,8263,8263,8263,8263,8263,8263,8263,8263,8127,8127,8127,8127,8127,8127,17738,17738,17738,17738,17738,17738,17738,17738,11422,11422,14098,14098,14098,3230,3230,3230,3230,3230,3230,3230,3230,3230,3230,3230,14465,14465,14465,6546,6546,6546,6546,8263,8263,8127,17738,17738,3230,3230,3230,8263,3230,8263,8127,17738,3230,8263,8127,17738,3230,8263,3230,8263,3230,8451,7537,8189,9040,8127,17738,11422,14098,3230,14465,6546,8451,7537,8263,17738,3230,⋯
2,13531,0,2738,2738,2738,2738,3158,3158,3158,3158,3938,3938,3981,3981,2492,2492,2492,2492,2492,2492,2492,2492,2492,2492,2492,3018,3018,3018,3018,3018,3018,5422,5422,5422,5422,5422,5422,5422,5422,2004,2004,3790,3790,3790,9749,9749,9749,9749,9749,9749,9749,9749,9749,9749,9749,6910,6910,6910,6161,6161,6161,6161,2492,2492,3018,5422,5422,9749,9749,9749,2492,9749,2492,3018,5422,9749,2492,3018,5422,9749,2492,9749,2492,9749,2738,3158,3938,3981,3018,5422,2004,3790,9749,6910,6161,2738,3158,2492,5422,9749,⋯
3,10963,3200,0,0,0,0,440,440,440,440,4199,4199,4242,4242,310,310,310,310,310,310,310,310,310,310,310,534,534,534,534,534,534,6252,6252,6252,6252,6252,6252,6252,6252,4184,4184,6673,6673,6673,5386,5386,5386,5386,5386,5386,5386,5386,5386,5386,5386,4479,4479,4479,6422,6422,6422,6422,310,310,534,6252,6252,5386,5386,5386,310,5386,310,534,6252,5386,310,534,6252,5386,310,5386,310,5386,0,440,4199,4242,534,6252,4184,6673,5386,4479,6422,0,440,310,6252,5386,⋯
4,10963,3200,0,0,0,0,440,440,440,440,4199,4199,4242,4242,310,310,310,310,310,310,310,310,310,310,310,534,534,534,534,534,534,6252,6252,6252,6252,6252,6252,6252,6252,4184,4184,6673,6673,6673,5386,5386,5386,5386,5386,5386,5386,5386,5386,5386,5386,4479,4479,4479,6422,6422,6422,6422,310,310,534,6252,6252,5386,5386,5386,310,5386,310,534,6252,5386,310,534,6252,5386,310,5386,310,5386,0,440,4199,4242,534,6252,4184,6673,5386,4479,6422,0,440,310,6252,5386,⋯
5,10963,3200,0,0,0,0,440,440,440,440,4199,4199,4242,4242,310,310,310,310,310,310,310,310,310,310,310,534,534,534,534,534,534,6252,6252,6252,6252,6252,6252,6252,6252,4184,4184,6673,6673,6673,5386,5386,5386,5386,5386,5386,5386,5386,5386,5386,5386,4479,4479,4479,6422,6422,

In [37]:
vehicle_type = vehicle_type[:,2:end]

Row,type
,Int64
1,1
2,1
3,2
4,1
5,2
6,1
7,2
8,1
9,1


In [38]:
(vehicle_depot_mapping[1,1] == 1)*2

2

### Defining Parameters

In [46]:
n_depots = 2
n_pickup = 70
n_destination = n_pickup
V = n_depots + n_pickup + n_destination    # Total locations

depots = [1: 1: n_depots;]
pickups = [1+n_depots: 1: n_depots+n_pickup;]
destinations = [1+n_depots+n_pickup: 1: n_depots+n_pickup+n_destination;];

In [47]:
depots

2-element Vector{Int64}:
 1
 2

In [48]:
pickups

70-element Vector{Int64}:
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
  ⋮
 61
 62
 63
 64
 65
 66
 67
 68
 69
 70
 71
 72

In [49]:
destinations

70-element Vector{Int64}:
  73
  74
  75
  76
  77
  78
  79
  80
  81
  82
  83
  84
  85
   ⋮
 131
 132
 133
 134
 135
 136
 137
 138
 139
 140
 141
 142

In [50]:
# n_vehicles_types = maximum(vehicle_type[!,1])
K = size(vehicle_type)[1] # n_vehicles

10

In [53]:
max_pickups_per_vehicle = 15

15

In [97]:
# # Build model.
# model = Model(Gurobi.Optimizer)
# # set_optimizer_attribute(model, "OutputFlag", solver_output)


# # Insert variable.
# # We create a huge number of variables because we map multiple locations with the same index.
# @variable(model, x[i=1:V, j=1:V, k=1:K], Bin)
# @variable(model, y[k=1:K], Bin)


# # Insert constraints.
# @constraint(model, [j=pickups[1]:pickups[n_pickup]], sum(sum(x[i,j,k] for i=1:V) for k=1:K) <= 1) # one patient is only picked up once
# @constraint(model, [i=pickups[1]:pickups[n_pickup]], sum(x[i,i+n_pickup,k] for k=1:K) == 1) # any 1 vehicles transports the patient from origin to destination
# # @constraint(model, [i=pickups[1]:pickups[n_pickup], k=1:K], sum(x[i,j,k] for j=1:V) - sum(x[i+n_pickup,j,k] for j=1:V) == 0)
# @constraint(model, [j=1:V, k=1:K], sum(x[i,j,k] for i=1:V) - sum(x[j,i,k] for i=1:V) == 0) # flow balance for each vehicle
# @constraint(model, [k=1:K], sum(x[depots[vehicle_depot_mapping[k,1]],j,k] for j=1:V) == y[k]) # 1 vehicle goes from the depot to only 1 patient
# @constraint(model, [k=1:K], sum(x[i, depots[vehicle_depot_mapping[k,1]],k] for i=1:V) == y[k]) # 1 vehicle goes from the depot to only 1 patient
# @constraint(model, [k=1:K], sum(sum(x[i,j,k] for j=1:V) for i=pickups[1]:pickups[n_pickup]) <= max_pickups_per_vehicle) # 1 vehicles is restricted to do a set number of pickups at max
# @constraint(model, [i=1:V, j=1:V, k=1:K], x[i,j,k] <= y[k]) # patient transferred only if vehicle k gets out of depot.



# # Objective.
# @objective(model, Min, 
#     sum(fixed_cost_vehicle_type_mapping[vehicle_type[k,1],1]*y[k] for k=1:K) +
#     sum(sum(sum(distance_cost_vehicle_type_mapping[vehicle_type[k,1],1]*distance_mapping[i,j]*x[i,j,k] for i=1:V) for j=1:V) for k=1:K)
# )


# # Optimize.
# optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 203230 rows, 201650 columns and 1006080 nonzeros
Model fingerprint: 0x44ebed8f
Variable types: 0 continuous, 201650 integer (201650 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 8785.6960000
Presolve removed 4240 rows and 1410 columns
Presolve time: 0.78s
Presolved: 198990 rows, 200240 columns, 996200 nonzeros
Variable types: 0 continuous, 200240 integer (200240 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 1.718808e+03, 1285 iterations, 0.63 seconds (1.

In [126]:
# Build model.
model = Model(Gurobi.Optimizer)
# set_optimizer_attribute(model, "OutputFlag", solver_output)


# Insert variable.
# We create a huge number of variables because we map multiple locations with the same index.
@variable(model, x[i=1:V, j=1:V, k=1:K], Bin)
@variable(model, y[k=1:K], Bin)
@variable(model, u[i=1:V, k=1:K], Int)


# Insert constraints.
@constraint(model, [i=pickups[1]:pickups[n_pickup]], sum(sum(x[i,j,k] for j=1:V) for k=1:K) <= 1) # one patient is only picked up once
@constraint(model, [i=pickups[1]:pickups[n_pickup]], sum(x[i,i+n_pickup,k] for k=1:K) == 1) # any 1 vehicles transports the patient from origin to destination
@constraint(model, [j=1:V, k=1:K], sum(x[i,j,k] for i=1:V) - sum(x[j,i,k] for i=1:V) == 0) # flow balance for each vehicle
@constraint(model, [k=1:K], sum(x[depots[vehicle_depot_mapping[k,1]],j,k] for j=pickups[1]:pickups[n_pickup]) == y[k]) # 1 vehicle goes from the depot to only 1 patient
@constraint(model, [k=1:K], sum(sum(x[i,j,k] for j=1:V) for i=pickups[1]:pickups[n_pickup]) <= max_pickups_per_vehicle) # 1 vehicles is restricted to do a set number of pickups at max
@constraint(model, [i=1:V, j=1:V, k=1:K], x[i,j,k] <= y[k]) # patient transferred only if vehicle k gets out of depot.
# @constraint(model, [k=1:K], u[1,k] == 1)
# @constraint(model, [i=2:V, k=1:K], 2 <= u[i,k] <= V)
# @constraint(model, [i=2:V, k=1:K], u[i,k] <= V)
# @constraint(model, [i=1:V, j=1:V, k=1:K], u[i,k] - u[j,k] + 1 <= (V-1)*(1-x[i,j,k]))


# Objective.
@objective(model, Min, 
    sum(fixed_cost_vehicle_type_mapping[vehicle_type[k,1],1]*y[k] for k=1:K) +
    sum(sum(sum(distance_cost_vehicle_type_mapping[vehicle_type[k,1],1]*distance_mapping[i,j]*x[i,j,k] for i=1:V) for j=1:V) for k=1:K)
)


# Optimize.
optimize!(model)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 203220 rows, 203070 columns and 1003930 nonzeros
Model fingerprint: 0x10ba7b89
Variable types: 0 continuous, 203070 integer (201650 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 100190 rows and 100840 columns
Presolve time: 0.52s
Presolved: 103030 rows, 102230 columns, 408900 nonzeros
Variable types: 0 continuous, 102230 integer (102230 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 1.759800e+03, 1413 iterations, 0.35 seconds (0.74 work units)

    Nodes    |    Current Nod

In [98]:
## Naive Approach

In [101]:
shuffle(depots)

2-element Vector{Int64}:
 2
 1

In [117]:
start_depot = shuffle(depots)[1]
chosen_vehicle = shuffle([1:1;size(vehicle_type)[1];])[1]
variable_cost = distance_cost_vehicle_type_mapping[vehicle_type[chosen_vehicle,1],1]

initial_cost = fixed_cost_vehicle_type_mapping[vehicle_type[chosen_vehicle,1],1] + distance_cost_vehicle_type_mapping[vehicle_type[chosen_vehicle,1],1]*distance_mapping[start_depot,pickups[1]]
cost = initial_cost    

184.51

In [118]:
for i=1:(size(pickups)[1]-1)
    cost = cost + variable_cost*(distance_mapping[pickups[i], destinations[i]] + distance_mapping[destinations[i], pickups[i+1]])
end

In [119]:
pickups[end]

72

In [120]:
cost = cost + distance_mapping[pickups[end], destinations[end]]

13908.890000000001

In [121]:
distance_mapping[pickups[end], destinations[end]]

5488

In [79]:
for i = 1:size(vehicle_type)[1]
    println(sum(value.(x[:,:,i])))
end

0.0
0.0
291.0
0.0
293.00000039553964
0.0
288.0
35.00000000037206
0.0
26.9999995778573


In [80]:
vehicle_3 = DataFrame(value.(x)[:,:,3], :auto);
vehicle_5 = DataFrame(value.(x)[:,:,5], :auto);
vehicle_7 = DataFrame(value.(x)[:,:,7], :auto);
vehicle_8 = DataFrame(value.(x)[:,:,8], :auto);
vehicle_10 = DataFrame(value.(x)[:,:,10], :auto);

In [81]:
sequence_mapping_3 = sort(Tuple.(findall(>=(1), Matrix(vehicle_3))))

291-element Vector{Tuple{Int64, Int64}}:
 (6, 76)
 (20, 90)
 (32, 102)
 (35, 105)
 (37, 107)
 (39, 109)
 (44, 114)
 (49, 119)
 (53, 123)
 (54, 124)
 (57, 127)
 (59, 129)
 (63, 133)
 ⋮
 (141, 93)
 (141, 100)
 (141, 111)
 (141, 128)
 (141, 132)
 (141, 135)
 (142, 81)
 (142, 83)
 (142, 104)
 (142, 112)
 (142, 129)
 (142, 138)

In [82]:
sequence_mapping_5 = sort(Tuple.(findall(>=(1), Matrix(vehicle_5))))

292-element Vector{Tuple{Int64, Int64}}:
 (9, 79)
 (12, 82)
 (14, 84)
 (29, 99)
 (30, 100)
 (33, 103)
 (34, 104)
 (47, 117)
 (48, 118)
 (50, 120)
 (51, 121)
 (64, 134)
 (66, 136)
 ⋮
 (141, 82)
 (141, 84)
 (141, 93)
 (141, 107)
 (141, 111)
 (141, 128)
 (142, 81)
 (142, 98)
 (142, 112)
 (142, 119)
 (142, 136)
 (142, 138)

In [83]:
sequence_mapping_7 = sort(Tuple.(findall(>=(1), Matrix(vehicle_7))))

288-element Vector{Tuple{Int64, Int64}}:
 (5, 75)
 (22, 92)
 (23, 93)
 (31, 101)
 (38, 108)
 (41, 111)
 (42, 112)
 (45, 115)
 (52, 122)
 (55, 125)
 (58, 128)
 (60, 130)
 (61, 131)
 ⋮
 (141, 82)
 (141, 100)
 (141, 111)
 (141, 114)
 (141, 128)
 (141, 134)
 (142, 73)
 (142, 83)
 (142, 110)
 (142, 119)
 (142, 129)
 (142, 136)

In [84]:
sequence_mapping_8 = sort(Tuple.(findall(>=(1), Matrix(vehicle_8))))

33-element Vector{Tuple{Int64, Int64}}:
 (7, 77)
 (8, 78)
 (10, 80)
 (11, 81)
 (16, 86)
 (17, 87)
 (24, 94)
 (27, 97)
 (36, 106)
 (43, 113)
 (46, 116)
 (73, 17)
 (73, 83)
 ⋮
 (86, 10)
 (87, 117)
 (97, 7)
 (106, 43)
 (113, 36)
 (114, 128)
 (116, 8)
 (117, 11)
 (126, 129)
 (128, 80)
 (129, 73)
 (135, 76)

In [85]:
sequence_mapping_10 = sort(Tuple.(findall(>=(1), Matrix(vehicle_10))))

25-element Vector{Tuple{Int64, Int64}}:
 (3, 73)
 (4, 74)
 (13, 83)
 (15, 85)
 (18, 88)
 (19, 89)
 (21, 91)
 (25, 95)
 (26, 96)
 (40, 110)
 (62, 132)
 (72, 142)
 (73, 21)
 (83, 19)
 (85, 4)
 (88, 118)
 (89, 26)
 (91, 40)
 (95, 62)
 (96, 3)
 (98, 15)
 (110, 18)
 (118, 13)
 (132, 72)
 (142, 25)

I just tried constrained the number of pickups to be constrained by 2 patients (since we only had 4 right now), and this is what we get! It choses 2 vehicles to do different patients!! 3-4 done by 5th vehicle and 5,6 done by 3rd vehicle and both are vehicles of type 2 (electric) so the choice of vehicles also works good

I also tried putting both the vehicles in different depots and it works. Just that they are currently going back to their original depot. Maybe we can say that it goes back to original, otherwise there would have been a complexity of capacity and vacancy at each depot, and thus to not do that, we put this constrain.

In [86]:
df_sequence_mapping = DataFrame(sequence_mapping)

Row,1,2
,Int64,Int64
1,2,25
2,3,73
3,4,74
4,8,78
5,15,85
6,24,94
7,25,95
8,26,96
9,28,98


In [87]:
CSV.write("../data/scaled_up_70/df_sequence_mapping_1.csv", DataFrame(sequence_mapping_3))
CSV.write("../data/scaled_up_70/df_sequence_mapping_2.csv", DataFrame(sequence_mapping_5))
CSV.write("../data/scaled_up_70/df_sequence_mapping_3.csv", DataFrame(sequence_mapping_7))
CSV.write("../data/scaled_up_70/df_sequence_mapping_4.csv", DataFrame(sequence_mapping_8))
CSV.write("../data/scaled_up_70/df_sequence_mapping_5.csv", DataFrame(sequence_mapping_10))

"../data/scaled_up_70/df_sequence_mapping_5.csv"